Detailed list of contributions:

Nour Belabed: 4.1 Joins and Subqueries |
Adam Khalil: inserts in tables and 4.2 Window functions |
Max Jin: Tables creation and 4.3 Triggers

In [1]:
import sqlite3

In [18]:
conn = sqlite3.connect('Assignment_database.db')
cursor = conn.cursor()
print("Opened database successfully")

Opened database successfully


We create the tables in our database

In [3]:
cursor.execute('''
    CREATE TABLE Employees (
        EmployeeID INTEGER PRIMARY KEY,
        Name TEXT,
        Position TEXT,
        DepartmentID INTEGER,
        Salary REAL
    )
''')

In [4]:
cursor.execute('''
    CREATE TABLE Departments (
        DepartmentID INTEGER PRIMARY KEY,
        DepartmentName TEXT
    )
''')

In [5]:
cursor.execute('''
    CREATE TABLE if not exists Projects (
        ProjectID INTEGER PRIMARY KEY,
        ProjectName TEXT,
        DepartmentID INTEGER
    )
''')

In [6]:
cursor.execute('''
    CREATE TABLE if not exists Assignments (
        EmployeeID INTEGER,
        ProjectID INTEGER,
        HoursWorked INTEGER,
        PRIMARY KEY (EmployeeID, ProjectID),
        FOREIGN KEY (EmployeeID) REFERENCES Employees(EmployeeID),
        FOREIGN KEY (ProjectID) REFERENCES Projects(ProjectID)
    )
''')

In [7]:
conn.commit()

We display the list of the tables so we check on the good execution of the previous queries.

In [8]:
# Connexion à la base de données
conn = sqlite3.connect('Assignment_database.db')
cursor = conn.cursor()

# Récupérer la liste de toutes les tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Afficher le nom de chaque table
for table in tables:
    print("Table:", table[0])

Table: Employees
Table: Departments
Table: Projects
Table: Assignments


We populate our tables by inserting data in them.

In [9]:
cursor.executemany('''
    INSERT INTO Employees (Name, Position, DepartmentID, Salary)
    VALUES (?, ?, ?, ?)
''', [
    ('John Doe', 'Software Engineer', 1, 80000),
    ('Jane Smith', 'Project Manager', 2, 90000),
    ('Bob Johnson', 'Data Scientist', 1, 85000),
    ('Alice Brown', 'Marketing Specialist', 3, 75000),
    ('Charlie Wilson', 'QA Tester', 2, 70000),
    ('Adam Khalil', 'Software Developer', 1, 92000),
    ('Max Jin', 'Product Manager', 2, 35000),
    ('Marc Zhou', 'UX Designer', 1, 12000),
    ('Dana White', 'Sales Representative', 3, 172000),
    ('Nour Belabed', 'data Analyst', 2, 125000)
])

In [10]:
cursor.executemany('''
    INSERT INTO Departments (DepartmentName)
    VALUES (?)
''', [
    ('Engineering',),
    ('Management',),
    ('Marketing',),
    ('Finance',),
    ('Human Resources',)
])

In [11]:
cursor.executemany('''
    INSERT INTO Projects (ProjectName, DepartmentID)
    VALUES (?, ?)
''', [
    ('Project A', 1),
    ('Project B', 2),
    ('Project C', 3),
    ('Project D', 1),
    ('Project E', 2)
])

In [12]:
cursor.executemany('''
    INSERT INTO Assignments (EmployeeID, ProjectID, HoursWorked)
    VALUES (?, ?, ?)
''', [
    (1, 1, 40),
    (2, 2, 35),
    (3, 1, 30),
    (4, 3, 25),
    (5, 2, 20),
    (6, 1, 40),
    (7, 2, 35),
    (8, 1, 30),
    (9, 3, 25),
    (10, 2, 20),
])

We create the query that lists all the employees with a select, then we add a join clause to display the names of the projects they are working on.

In [13]:
cursor.execute('''
    SELECT
        E.EmployeeID,
        E.Name AS EmployeeName,
        A.ProjectID,
        P.ProjectName
    FROM
        Employees E
        JOIN Assignments A ON E.EmployeeID = A.EmployeeID
        JOIN Projects P ON A.ProjectID = P.ProjectID
''')

# Fetch all the results
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)

(1, 'John Doe', 1, 'Project A')
(2, 'Jane Smith', 2, 'Project B')
(3, 'Bob Johnson', 1, 'Project A')
(4, 'Alice Brown', 3, 'Project C')
(5, 'Charlie Wilson', 2, 'Project B')
(6, 'Adam Khalil', 1, 'Project A')
(7, 'Max Jin', 2, 'Project B')
(8, 'Marc Zhou', 1, 'Project A')
(9, 'Dana White', 3, 'Project C')
(10, 'Nour Belabed', 2, 'Project B')


We create a query to calculate the average salary of each department, then we group them by the department name, we order it in descending and we limit the result to 1 value so we only keep the highest average.

In [14]:
cursor.execute('''
    SELECT DepartmentName, AVG(Salary) AS AvgSalary
    FROM Departments
    JOIN Employees ON Departments.DepartmentID = Employees.DepartmentID
    GROUP BY DepartmentName
    ORDER BY AvgSalary DESC
    LIMIT 1
''')

# Fetch the result
result = cursor.fetchone()
print("Department with the highest average salary:")
print("Department Name:", result[0])
print("Average Salary:", result[1])

Department with the highest average salary:
Department Name: Marketing
Average Salary: 123500.0


We create a query with the rank() function to rank the employees based on their salary in a descending order for each department (from the first to the third)

In [19]:
cursor.execute('''
    SELECT
    EmployeeID,
    Name,
    Position,
    DepartmentID,
    Salary,
    RANK() OVER (PARTITION BY DepartmentID ORDER BY Salary DESC) AS SalaryRank
FROM
    Employees;

''')
results = cursor.fetchall()
for row in results:
    print(row)

(6, 'Adam Khalil', 'Software Developer', 1, 92000.0, 1)
(3, 'Bob Johnson', 'Data Scientist', 1, 85000.0, 2)
(1, 'John Doe', 'Software Engineer', 1, 80000.0, 3)
(8, 'Marc Zhou', 'UX Designer', 1, 12000.0, 4)
(10, 'Nour Belabed', 'data Analyst', 2, 125000.0, 1)
(2, 'Jane Smith', 'Project Manager', 2, 90000.0, 2)
(5, 'Charlie Wilson', 'QA Tester', 2, 70000.0, 3)
(7, 'Max Jin', 'Product Manager', 2, 35000.0, 4)
(9, 'Dana White', 'Sales Representative', 3, 172000.0, 1)
(4, 'Alice Brown', 'Marketing Specialist', 3, 75000.0, 2)


Finally, we create a trigger that activates automatically when an assignment is added, so it updates the total hours worked in a project.

In [16]:
cursor.execute('''
    CREATE TRIGGER update_total_hours
AFTER INSERT ON Assignments
FOR EACH ROW
BEGIN
    UPDATE Projects
    SET TotalHoursWorked = TotalHoursWorked + NEW.HoursWorked
    WHERE ProjectID = NEW.ProjectID;
END;
''')
# Commit and close the connection
conn.commit()
conn.close()